<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [20]:
import pandas as pd
import numpy as np


df = pd.read_csv("https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv")

In [21]:
pd.set_option('display.max_columns', None)

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [22]:
df.shape

(7043, 21)

In [23]:
df.isnull().any()

customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges        False
Churn               False
dtype: bool

In [24]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,1261-FWTTE,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,20.2,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [25]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [26]:
df = df.drop(['customerID'], axis=1)
df['TotalCharges'] = df['TotalCharges'].str.strip()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
df['TotalCharges'].fillna(0, inplace=True)

In [27]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [28]:
df.isnull().sum()


gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df['Churn'], test_size=0.25, random_state=1337, stratify=df['Churn'])

In [30]:
y_train

3388     No
4731     No
1021     No
6968    Yes
6361     No
       ... 
5949     No
2841     No
3161     No
5123    Yes
4300     No
Name: Churn, Length: 5282, dtype: object

In [31]:
y_test

541      No
1737     No
1483     No
2523     No
2370     No
       ... 
5686    Yes
1198     No
2090     No
5669     No
616      No
Name: Churn, Length: 1761, dtype: object

In [32]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

encoder = ce.OneHotEncoder()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [33]:
X_train.head()

,gender_1,gender_2,SeniorCitizen,Partner_1,Partner_2,Dependents_1,Dependents_2,tenure,PhoneService_1,PhoneService_2,MultipleLines_1,MultipleLines_2,MultipleLines_3,InternetService_1,InternetService_2,InternetService_3,OnlineSecurity_1,OnlineSecurity_2,OnlineSecurity_3,OnlineBackup_1,OnlineBackup_2,OnlineBackup_3,DeviceProtection_1,DeviceProtection_2,DeviceProtection_3,TechSupport_1,TechSupport_2,TechSupport_3,StreamingTV_1,StreamingTV_2,StreamingTV_3,StreamingMovies_1,StreamingMovies_2,StreamingMovies_3,Contract_1,Contract_2,Contract_3,PaperlessBilling_1,PaperlessBilling_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,MonthlyCharges,TotalCharges
3388,1,0,0,1,0,1,0,65,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,55.15,3673.15
4731,0,1,0,1,0,0,1,19,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,19.90,357.70
1021,0,1,0,1,0,1,0,28,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,54.35,1426.45
6968,1,0,1,0,1,1,0,22,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,74.40,1692.60
6361,1,0,0,0,1,1,0,7,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,62.80,418.30


In [34]:
X_train.values.shape


(5282, 45)

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Trying logistic regression for baseline.

In [51]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

lr.score(X_test, y_test)


0.8103350369108461

In [39]:
X_train



array([[ 1.0045541 , -1.0045541 , -0.4449765 , ..., -0.52491648,
        -0.31170952,  0.62941018],
       [-0.99546654,  0.99546654, -0.4449765 , ..., -0.52491648,
        -1.48499902, -0.84344155],
       [-0.99546654,  0.99546654, -0.4449765 , ..., -0.52491648,
        -0.33833737, -0.36866127],
       ...,
       [-0.99546654,  0.99546654, -0.4449765 , ..., -0.52491648,
        -1.50330567, -0.85805701],
       [ 1.0045541 , -1.0045541 ,  2.24730971, ...,  1.90506498,
         0.35897937, -0.49260391],
       [ 1.0045541 , -1.0045541 , -0.4449765 , ..., -0.52491648,
        -1.32856042, -0.73347022]])

In [60]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

n_inputs = X_train.shape[1]

model = Sequential()
model.add(Dense(128, activation='sigmoid', input_dim=(n_inputs)))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])

In [61]:
model.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/10
529/529 [==============================] - 1s 2ms/step - loss: 0.4608 - accuracy: 0.7715 - val_loss: 0.4226 - val_accuracy: 0.7978

In [ ]:
df['Churn'].value_counts(normalize=True)

In [ ]:
# def create_model():
#     model = Sequential()
#     model.add(Dense(128, activation='sigmoid', input_dim=(n_inputs)))
#     model.add(Dense(128, activation='sigmoid'))
#     model.add(Dense(1, activation='sigmoid'))
#
#     return model
#
# param_grid = {'batch_size': [2, 4, 6, 8, 10, 20, 32],
#               ''}



In [65]:
from kerastuner import HyperModel

class MultilayerPerceptron(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int('units', min_value=16, max_value=256, step=32, default=128)))
        model.add(Dense(units=hp.Int('units', min_value=16, max_value=256, step=32, default=128)))
        model.add(Dense(self.num_classes, activation='sigmoid'))

        model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2,
                                                               sampling='LOG', default=1e-3)),
                      loss='BinaryCrossentropy', metrics=['accuracy'])

        return model

In [66]:
from kerastuner.tuners import RandomSearch

INPUT_SHAPE = (5282, 45)
NUM_CLASSES = 1

hypermodel = MultilayerPerceptron(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(hypermodel, objective='val_accuracy', seed=1337, max_trials=10, executions_per_trial=2,
                     directory='random_search')

In [67]:
tuner.search_space_summary()

[Search space summary]
 |-Default search space size: 2
 > units (Int)
 |-default: 128
 |-max_value: 256
 |-min_value: 16
 |-sampling: None
 |-step: 32
 > learning_rate (Float)
 |-default: 0.001
 |-max_value: 0.01
 |-min_value: 0.0001
 |-sampling: log
 |-step: None


In [68]:
tuner.search(X_train, y_train, epochs=20, validation_split=0.1)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

149/149 [==============================] - ETA: 0s - loss: 0.8671 - accuracy: 0.3750 - ETA: 0s - loss: 0.6357 - accuracy: 0.7302 - ETA: 0s - loss: 0.5609 - accuracy: 0.7458 - ETA: 0s - loss: 0.5319 - accuracy: 0.7497

In [69]:
tuner.results_summary()

[Results summary]
 |-Results in random_search\untitled_project
 |-Showing 10 best trials
 |-Objective(name='val_accuracy', direction='max')
[Trial summary]
 |-Trial ID: f294b28ecef49da87ea6e105903c53e0
 |-Score: 0.8393194675445557
 |-Best step: 0
 > Hyperparameters:
 |-learning_rate: 0.0005065312024231252
 |-units: 176
[Trial summary]
 |-Trial ID: 0c956b8f4ed3be6f045fc926945b2838
 |-Score: 0.8383742868900299
 |-Best step: 0
 > Hyperparameters:
 |-learning_rate: 0.005421195615159703
 |-units: 176
[Trial summary]
 |-Trial ID: d86349ad6b222553d23481975641d059
 |-Score: 0.8374291062355042
 |-Best step: 0
 > Hyperparameters:
 |-learning_rate: 0.0006818920464866776
 |-units: 112
[Trial summary]
 |-Trial ID: 8934a91ca4e9ad35600ba7561db189cb
 |-Score: 0.8374291062355042
 |-Best step: 0
 > Hyperparameters:
 |-learning_rate: 0.000396478365712187
 |-units: 48
[Trial summary]
 |-Trial ID: f9f0096b846e6ee52bfc849376930a52
 |-Score: 0.8355387449264526
 |-Best step: 0
 > Hyperparameters:
 |-learning_

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?